In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import gdown
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import rioxarray as rio
import rasterstats
from pprint import pprint

# import geoprofile
import micro
import gtsa

## Download sample data

In [3]:
url = 'https://drive.google.com/drive/u/1/folders/1E1FbgeS2YHqnZOsXpbXTIpWhWT1oDR5N'
k13_dem_folder='../data/dems/k13_coreg/'
# gdown.download_folder(url, output=k13_dem_folder, quiet=False, use_cookies=False)

url = 'https://drive.google.com/drive/u/1/folders/1NBJ87RWqVEixA7XuN4uvmtjGJ03JIU56'
m14_dem_folder='../data/dems/m14_coreg/'
# gdown.download_folder(url, output=m14_dem_folder, quiet=False, use_cookies=False)

In [4]:
!tree ../data/dems

../data/dems
├── k13
│   ├── dsyr1
│   │   ├── K13_lidar_20210604_cc.tif
│   │   ├── K13_lidar_20210913_cc.tif
│   │   ├── K13_sfm_20210604_cc.tif
│   │   ├── K13_sfm_20210819_cc.tif
│   │   └── K13_sfm_20210913_cc.tif
│   ├── wsyr1
│   │   ├── K13_lidar_20201109_cc.tif
│   │   ├── K13_lidar_20210513_cc.tif
│   │   ├── K13_sfm_20201109_cc.tif
│   │   ├── K13_sfm_20210208_cc.tif
│   │   ├── K13_sfm_20210406_cc.tif
│   │   └── K13_sfm_20210513_cc.tif
│   └── wsyr2
│       ├── K13_lidar_20211007_cc.tif
│       ├── K13_lidar_20220531_cc.tif
│       ├── K13_sfm_20211007_cc.tif
│       ├── K13_sfm_20220208_cc.tif
│       ├── K13_sfm_20220503_cc.tif
│       └── K13_sfm_20220531_cc.tif
├── k13_coreg
│   └── KID-13
│       ├── dsyr1
│       │   ├── k13_20210604_lidar_coreg.nc
│       │   ├── k13_20210604_lidar_coreg.tif
│       │   ├── k13_20210605_sfm_coreg.nc
│       │   ├── k13_20210605_sfm_coreg.tif
│       │   ├── k13_20210819_sfm_coreg.nc
│       │   ├── k13_20210819_sfm_coreg.tif
│      

In [5]:
k13_wsyr1_dem_files = sorted(Path(k13_dem_folder).glob('KID-13/wsyr1/*.tif'))
k13_wsyr1_dem_files = [i.as_posix() for i in k13_wsyr1_dem_files]

k13_dsyr1_dem_files = sorted(Path(k13_dem_folder).glob('KID-13/dsyr1/*.tif'))
k13_dsyr1_dem_files = [i.as_posix() for i in k13_dsyr1_dem_files]

k13_wsyr2_dem_files = sorted(Path(k13_dem_folder).glob('KID-13/wsyr2/*.tif'))
k13_wsyr2_dem_files = [i.as_posix() for i in k13_wsyr2_dem_files]

m14_wsyr1_dem_files = sorted(Path(m14_dem_folder).glob('MEL-14/wsyr1/*.tif'))
m14_wsyr1_dem_files = [i.as_posix() for i in m14_wsyr1_dem_files]

m14_dsyr1_dem_files = sorted(Path(m14_dem_folder).glob('MEL-14/dsyr1/*.tif'))
m14_dsyr1_dem_files = [i.as_posix() for i in m14_dsyr1_dem_files]

m14_wsyr2_dem_files = sorted(Path(m14_dem_folder).glob('MEL-14/wsyr2/*.tif'))
m14_wsyr2_dem_files = [i.as_posix() for i in m14_wsyr2_dem_files]

In [6]:
k13_wsyr1_dem_files, k13_dsyr1_dem_files, k13_wsyr2_dem_files, \
m14_wsyr1_dem_files, m14_dsyr1_dem_files, m14_wsyr2_dem_files

(['../data/dems/k13_coreg/KID-13/wsyr1/k13_20201109_lidar_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20201110_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20210208_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20210406_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20210513_sfm_coreg.tif'],
 ['../data/dems/k13_coreg/KID-13/dsyr1/k13_20210604_lidar_coreg.tif',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210605_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210819_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210913_sfm_coreg.tif'],
 ['../data/dems/k13_coreg/KID-13/wsyr2/k13_20211007_lidar_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20211008_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20220208_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20220503_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20220531_sfm_coreg.tif'],
 ['../data/dems/m14_coreg/MEL-14/wsyr1/

## Select analysis ready data
- Assuming SfM and lidar collected on the same date are only used for validation of the SfM data
- Assuming the lidar is always "better" and will thus be used for the quantitative measurements and time series analysis

In [7]:
# KID-13
dsm_files_k13_wsyr1 = ['../data/dems/k13_coreg/KID-13/wsyr1/k13_20201109_lidar_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20201110_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20210208_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20210406_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20210513_sfm_coreg.tif']

dsm_files_k13_dsyr1 = ['../data/dems/k13_coreg/KID-13/dsyr1/k13_20210604_lidar_coreg.tif',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210605_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210819_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210913_sfm_coreg.tif']

dsm_files_k13_wsyr2 = ['../data/dems/k13_coreg/KID-13/wsyr2/k13_20211007_lidar_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20211008_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20220208_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20220503_sfm_coreg.tif',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20220531_sfm_coreg.tif']


# MEL-14
dsm_files_m14_wsyr1 = ['../data/dems/m14_coreg/MEL-14/wsyr1/m14_20201203_lidar_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/wsyr1/m14_20201204_sfm_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/wsyr1/m14_20210224_sfm_coreg.tif']

dsm_files_m14_dsyr1 = ['../data/dems/m14_coreg/MEL-14/dsyr1/m14_20210603_lidar_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/dsyr1/m14_20210604_sfm_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/dsyr1/m14_20210914_sfm_coreg.tif']

dsm_files_m14_wsyr2 = ['../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220309_lidar_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220310_sfm_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220316_sfm_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220324_sfm_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220411_sfm_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220428_sfm_coreg.tif',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220601_sfm_coreg.tif']

## Parse time stamps

#### KID-13 WSYR1

In [8]:
strings_k13_wsyr1 = gtsa.io.parse_timestamps(dsm_files_k13_wsyr1, date_string_pattern='_........_')
date_strings_k13_wsyr1 = [x[1:-1] for x in strings_k13_wsyr1]

# ensure chronological sorting
date_strings_k13_wsyr1, dsm_files_k13_wsyr1 = list(zip(*sorted(zip(date_strings_k13_wsyr1, dsm_files_k13_wsyr1))))
date_times_k13_wsyr1 = [pd.to_datetime(x, format='%Y%m%d') for x in date_strings_k13_wsyr1]

list(zip(date_strings_k13_wsyr1, dsm_files_k13_wsyr1))

[('20201109',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20201109_lidar_coreg.tif'),
 ('20201110',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20201110_sfm_coreg.tif'),
 ('20210208',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20210208_sfm_coreg.tif'),
 ('20210406',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20210406_sfm_coreg.tif'),
 ('20210513',
  '../data/dems/k13_coreg/KID-13/wsyr1/k13_20210513_sfm_coreg.tif')]

#### KID-13 DSYR1

In [9]:
strings_k13_dsyr1 = gtsa.io.parse_timestamps(dsm_files_k13_dsyr1, date_string_pattern='_........_')
date_strings_k13_dsyr1 = [x[1:-1] for x in strings_k13_dsyr1]

# ensure chronological sorting
date_strings_k13_dsyr1, dsm_files_k13_dsyr1 = list(zip(*sorted(zip(date_strings_k13_dsyr1, dsm_files_k13_dsyr1))))
date_times_k13_dsyr1 = [pd.to_datetime(x, format='%Y%m%d') for x in date_strings_k13_dsyr1]

list(zip(date_strings_k13_dsyr1, dsm_files_k13_dsyr1))

[('20210604',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210604_lidar_coreg.tif'),
 ('20210605',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210605_sfm_coreg.tif'),
 ('20210819',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210819_sfm_coreg.tif'),
 ('20210913',
  '../data/dems/k13_coreg/KID-13/dsyr1/k13_20210913_sfm_coreg.tif')]

#### KID-13 WSYR2

In [10]:
strings_k13_wsyr2 = gtsa.io.parse_timestamps(dsm_files_k13_wsyr2, date_string_pattern='_........_')
date_strings_k13_wsyr2 = [x[1:-1] for x in strings_k13_wsyr2]

# ensure chronological sorting
date_strings_k13_wsyr2, dsm_files_k13_wsyr2 = list(zip(*sorted(zip(date_strings_k13_wsyr2, dsm_files_k13_wsyr2))))
date_times_k13_wsyr2 = [pd.to_datetime(x, format='%Y%m%d') for x in date_strings_k13_wsyr2]

list(zip(date_strings_k13_wsyr2, dsm_files_k13_wsyr2))

[('20211007',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20211007_lidar_coreg.tif'),
 ('20211008',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20211008_sfm_coreg.tif'),
 ('20220208',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20220208_sfm_coreg.tif'),
 ('20220503',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20220503_sfm_coreg.tif'),
 ('20220531',
  '../data/dems/k13_coreg/KID-13/wsyr2/k13_20220531_sfm_coreg.tif')]

#### MEL-14 WSYR1

In [11]:
strings_m14_wsyr1 = gtsa.io.parse_timestamps(dsm_files_m14_wsyr1, date_string_pattern='_........_')
date_strings_m14_wsyr1 = [x[1:-1] for x in strings_m14_wsyr1]

# ensure chronological sorting
date_strings_m14_wsyr1, dsm_files_m14_wsyr1 = list(zip(*sorted(zip(date_strings_m14_wsyr1, dsm_files_m14_wsyr1))))
date_times_m14_wsyr1 = [pd.to_datetime(x, format='%Y%m%d') for x in date_strings_m14_wsyr1]

list(zip(date_strings_m14_wsyr1, dsm_files_m14_wsyr1))

[('20201203',
  '../data/dems/m14_coreg/MEL-14/wsyr1/m14_20201203_lidar_coreg.tif'),
 ('20201204',
  '../data/dems/m14_coreg/MEL-14/wsyr1/m14_20201204_sfm_coreg.tif'),
 ('20210224',
  '../data/dems/m14_coreg/MEL-14/wsyr1/m14_20210224_sfm_coreg.tif')]

#### MEL-14 DSYR1

In [12]:
strings_m14_dsyr1 = gtsa.io.parse_timestamps(dsm_files_m14_dsyr1, date_string_pattern='_........_')
date_strings_m14_dsyr1 = [x[1:-1] for x in strings_m14_dsyr1]

# ensure chronological sorting
date_strings_m14_dsyr1, dsm_files_m14_dsyr1 = list(zip(*sorted(zip(date_strings_m14_dsyr1, dsm_files_m14_dsyr1))))
date_times_m14_dsyr1 = [pd.to_datetime(x, format='%Y%m%d') for x in date_strings_m14_dsyr1]

list(zip(date_strings_m14_dsyr1, dsm_files_m14_dsyr1))

[('20210603',
  '../data/dems/m14_coreg/MEL-14/dsyr1/m14_20210603_lidar_coreg.tif'),
 ('20210604',
  '../data/dems/m14_coreg/MEL-14/dsyr1/m14_20210604_sfm_coreg.tif'),
 ('20210914',
  '../data/dems/m14_coreg/MEL-14/dsyr1/m14_20210914_sfm_coreg.tif')]

#### MEL-14 WSYR2

In [13]:
strings_m14_wsyr2 = gtsa.io.parse_timestamps(dsm_files_m14_wsyr2, date_string_pattern='_........_')
date_strings_m14_wsyr2 = [x[1:-1] for x in strings_m14_wsyr2]

# ensure chronological sorting
date_strings_m14_wsyr2, dsm_files_m14_wsyr2 = list(zip(*sorted(zip(date_strings_m14_wsyr2, dsm_files_m14_wsyr2))))
date_times_m14_wsyr2 = [pd.to_datetime(x, format='%Y%m%d') for x in date_strings_m14_wsyr2]

list(zip(date_strings_m14_wsyr2, dsm_files_m14_wsyr2))

[('20220309',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220309_lidar_coreg.tif'),
 ('20220310',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220310_sfm_coreg.tif'),
 ('20220316',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220316_sfm_coreg.tif'),
 ('20220324',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220324_sfm_coreg.tif'),
 ('20220411',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220411_sfm_coreg.tif'),
 ('20220428',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220428_sfm_coreg.tif'),
 ('20220601',
  '../data/dems/m14_coreg/MEL-14/wsyr2/m14_20220601_sfm_coreg.tif')]

### Shapefiles for road clip

In [14]:
k13_mask_path = '../data/vectors/kid_mask/roadClip/KID13_roadClip.shp'
m14_mask_path = '../data/vectors/mel_mask/roadClip/MEL14_roadClip.shp'

# grab shapefile crs to write to rasters
k13_crs = gpd.read_file(k13_mask_path).crs
k13_mask_gdf = gpd.read_file(k13_mask_path)

m14_crs = gpd.read_file(m14_mask_path).crs
m14_mask_gdf = gpd.read_file(m14_mask_path)

## Create stacks
- resample to reference DEM grid and save as netcdf

In [15]:
reference_dem_k13_wsyr1 = dsm_files_k13_wsyr1[0]

ds_k13_wsyr1 = gtsa.io.xr_stack_geotifs(
    dsm_files_k13_wsyr1,
    date_times_k13_wsyr1,
    reference_dem_k13_wsyr1,
    resampling='cubic',
    save_to_nc=True,
    overwrite=True,
    cleanup=False,
    verbose=True,
)

ds_k13_wsyr1 = ds_k13_wsyr1.rio.clip(k13_mask_gdf["geometry"])

Resampled 5 of 5 dems to match reference DEM spatial_ref, crs, transform, bounds, and resolution.
Reading files from ../data/dems/k13_coreg/KID-13/wsyr1


In [16]:
reference_dem_k13_dsyr1 = dsm_files_k13_dsyr1[0]

ds_k13_dsyr1 = gtsa.io.xr_stack_geotifs(
    dsm_files_k13_dsyr1,
    date_times_k13_dsyr1,
    reference_dem_k13_dsyr1,
    resampling='cubic',
    save_to_nc=True,
    overwrite=True,
    cleanup=False,
    verbose=True,
)

ds_k13_dsyr1 = ds_k13_dsyr1.rio.clip(k13_mask_gdf["geometry"])

Resampled 4 of 4 dems to match reference DEM spatial_ref, crs, transform, bounds, and resolution.
Reading files from ../data/dems/k13_coreg/KID-13/dsyr1


In [17]:
reference_dem_k13_wsyr2 = dsm_files_k13_wsyr2[0]

ds_k13_wsyr2 = gtsa.io.xr_stack_geotifs(
    dsm_files_k13_wsyr2,
    date_times_k13_wsyr2,
    reference_dem_k13_wsyr2,
    resampling='cubic',
    save_to_nc=True,
    overwrite=True,
    cleanup=False,
    verbose=True,
)

ds_k13_wsyr2 = ds_k13_wsyr2.rio.clip(k13_mask_gdf["geometry"])

Resampled 5 of 5 dems to match reference DEM spatial_ref, crs, transform, bounds, and resolution.
Reading files from ../data/dems/k13_coreg/KID-13/wsyr2


In [18]:
reference_dem_m14_wsyr1 = dsm_files_m14_wsyr1[0]

ds_m14_wsyr1 = gtsa.io.xr_stack_geotifs(
    dsm_files_m14_wsyr1,
    date_times_m14_wsyr1,
    reference_dem_m14_wsyr1,
    resampling='cubic',
    save_to_nc=True,
    overwrite=True,
    cleanup=False,
    verbose=True,
)

ds_m14_wsyr1 = ds_m14_wsyr1.rio.clip(m14_mask_gdf["geometry"])

Resampled 3 of 3 dems to match reference DEM spatial_ref, crs, transform, bounds, and resolution.
Reading files from ../data/dems/m14_coreg/MEL-14/wsyr1


In [19]:
reference_dem_m14_dsyr1 = dsm_files_m14_dsyr1[0]

ds_m14_dsyr1 = gtsa.io.xr_stack_geotifs(
    dsm_files_m14_dsyr1,
    date_times_m14_dsyr1,
    reference_dem_m14_dsyr1,
    resampling='cubic',
    save_to_nc=True,
    overwrite=True,
    cleanup=False,
    verbose=True,
)

ds_m14_dsyr1 = ds_m14_dsyr1.rio.clip(m14_mask_gdf["geometry"])

Resampled 3 of 3 dems to match reference DEM spatial_ref, crs, transform, bounds, and resolution.
Reading files from ../data/dems/m14_coreg/MEL-14/dsyr1


In [20]:
reference_dem_m14_wsyr2 = dsm_files_m14_wsyr2[0]

ds_m14_wsyr2 = gtsa.io.xr_stack_geotifs(
    dsm_files_m14_wsyr2,
    date_times_m14_wsyr2,
    reference_dem_m14_wsyr2,
    resampling='cubic',
    save_to_nc=True,
    overwrite=True,
    cleanup=False,
    verbose=True,
)

ds_m14_wsyr2 = ds_m14_wsyr2.rio.clip(m14_mask_gdf["geometry"])

Resampled 7 of 7 dems to match reference DEM spatial_ref, crs, transform, bounds, and resolution.
Reading files from ../data/dems/m14_coreg/MEL-14/wsyr2


## Ensure `nodata` values are correct

In [21]:
# nd = np.nanmax(ds['band1'].isel(time=2).values)
# nd = np.nanmin(ds['band1'].isel(time=2).values)
# # ds_nodata = ds_nodata.where(ds_nodata!=nd)

# print(f"nodata: {ds_m14_wsyr2['band1'].rio.nodata}")
# print(f"encoded_nodata: {ds_m14_wsyr2['band1'].rio.encoded_nodata}")

In [22]:
# ds['band1'].rio.write_nodata(-9999, encoded=False, inplace=True)


# print(f"nodata: {ds['band1'].rio.nodata}")
# print(f"encoded_nodata: {ds['band1'].rio.encoded_nodata}")

## Save rasters for further analysis

In [23]:
# for i in ds_k13_wsyr1.data_vars:
#                 try:
#                     del ds_k13_wsyr1[i].attrs["grid_mapping"]
#                 except:
#                     pass

# ds_k13_wsyr1.isel(time=0).rio.to_raster('../../ll_micro/k13/wsyr1/k13_20201109_lidar_coreg.tif')
# ds_k13_wsyr1.isel(time=1).rio.to_raster('../../ll_micro/k13/wsyr1/k13_20201109_sfm_coreg.tif')
# ds_k13_wsyr1.isel(time=2).rio.to_raster('../../ll_micro/k13/wsyr1/k13_20210208_sfm_coreg.tif')
# ds_k13_wsyr1.isel(time=3).rio.to_raster('../../ll_micro/k13/wsyr1/k13_20210406_sfm_coreg.tif')
# ds_k13_wsyr1.isel(time=4).rio.to_raster('../../ll_micro/k13/wsyr1/k13_20210513_sfm_coreg.tif')

In [24]:
# for i in ds_k13_dsyr1.data_vars:
#                 try:
#                     del ds_k13_dsyr1[i].attrs["grid_mapping"]
#                 except:
#                     pass

# ds_k13_dsyr1.isel(time=0).rio.to_raster('../../ll_micro/k13/dsyr1/k13_20210604_lidar_coreg.tif')
# ds_k13_dsyr1.isel(time=1).rio.to_raster('../../ll_micro/k13/dsyr1/k13_20210604_sfm_coreg.tif')
# ds_k13_dsyr1.isel(time=2).rio.to_raster('../../ll_micro/k13/dsyr1/k13_20210819_sfm_coreg.tif')
# ds_k13_dsyr1.isel(time=3).rio.to_raster('../../ll_micro/k13/dsyr1/k13_20210913_sfm_coreg.tif')

In [25]:
# for i in ds_k13_wsyr2.data_vars:
#                 try:
#                     del ds_k13_wsyr2[i].attrs["grid_mapping"]
#                 except:
#                     pass

# ds_k13_wsyr2.isel(time=0).rio.to_raster('../../ll_micro/k13/wsyr2/k13_20211007_lidar_coreg.tif')
# ds_k13_wsyr2.isel(time=1).rio.to_raster('../../ll_micro/k13/wsyr2/k13_20211007_sfm_coreg.tif')
# ds_k13_wsyr2.isel(time=2).rio.to_raster('../../ll_micro/k13/wsyr2/k13_20220208_sfm_coreg.tif')
# ds_k13_wsyr2.isel(time=3).rio.to_raster('../../ll_micro/k13/wsyr2/k13_20220503_sfm_coreg.tif')
# ds_k13_wsyr2.isel(time=4).rio.to_raster('../../ll_micro/k13/wsyr2/k13_20220531_sfm_coreg.tif')

In [26]:
# for i in ds_m14_wsyr1.data_vars:
#                 try:
#                     del ds_m14_wsyr1[i].attrs["grid_mapping"]
#                 except:
#                     pass

# ds_m14_wsyr1.isel(time=0).rio.to_raster('../../ll_micro/m14/wsyr1/m14_20201203_lidar_coreg.tif')
# ds_m14_wsyr1.isel(time=1).rio.to_raster('../../ll_micro/m14/wsyr1/m14_20201203_sfm_coreg.tif')
# ds_m14_wsyr1.isel(time=2).rio.to_raster('../../ll_micro/m14/wsyr1/m14_20210224_sfm_coreg.tif')

In [27]:
# for i in ds_m14_dsyr1.data_vars:
#                 try:
#                     del ds_m14_dsyr1[i].attrs["grid_mapping"]
#                 except:
#                     pass

# ds_m14_dsyr1.isel(time=0).rio.to_raster('../../ll_micro/m14/dsyr1/m14_20210603_lidar_coreg.tif')
# ds_m14_dsyr1.isel(time=1).rio.to_raster('../../ll_micro/m14/dsyr1/m14_20210603_sfm_coreg.tif')
# ds_m14_dsyr1.isel(time=2).rio.to_raster('../../ll_micro/m14/dsyr1/m14_20210914_sfm_coreg.tif')

In [28]:
# for i in ds_m14_wsyr2.data_vars:
#                 try:
#                     del ds_m14_wsyr2[i].attrs["grid_mapping"]
#                 except:
#                     pass

# ds_m14_wsyr2.isel(time=0).rio.to_raster('../../ll_micro/m14/wsyr2/m14_20220309_lidar_coreg.tif')
# ds_m14_wsyr2.isel(time=1).rio.to_raster('../../ll_micro/m14/wsyr2/m14_20220309_sfm_coreg.tif')
# ds_m14_wsyr2.isel(time=2).rio.to_raster('../../ll_micro/m14/wsyr2/m14_20220316_sfm_coreg.tif')
# ds_m14_wsyr2.isel(time=3).rio.to_raster('../../ll_micro/m14/wsyr2/m14_20220324_sfm_coreg.tif')
# ds_m14_wsyr2.isel(time=4).rio.to_raster('../../ll_micro/m14/wsyr2/m14_20220411_sfm_coreg.tif')
# ds_m14_wsyr2.isel(time=5).rio.to_raster('../../ll_micro/m14/wsyr2/m14_20220428_sfm_coreg.tif')
# ds_m14_wsyr2.isel(time=6).rio.to_raster('../../ll_micro/m14/wsyr2/m14_20220601_sfm_coreg.tif')

## Plot first time stamp

In [29]:
# ds_k13_wsyr1['band1'].isel(time=0).plot(aspect='equal', size=5)
# plt.show();

# ds_k13_dsyr1['band1'].isel(time=0).plot(aspect='equal', size=5)
# plt.show();

# ds_k13_wsyr2['band1'].isel(time=0).plot(aspect='equal', size=5)
# plt.show();

# ds_m14_wsyr1['band1'].isel(time=0).plot(aspect='equal', size=5)
# plt.show();

# ds_m14_dsyr1['band1'].isel(time=0).plot(aspect='equal', size=5)
# plt.show();

# ds_m14_wsyr2['band1'].isel(time=0).plot(aspect='equal', size=5)
# plt.show();

## Compute differences for stack

In [30]:
ds_diff_k13_wsyr1_lidar = ds_k13_wsyr1['band1']- ds_k13_wsyr1['band1'].isel(time=0)
ds_diff_k13_wsyr1_sfm = ds_k13_wsyr1['band1']- ds_k13_wsyr1['band1'].isel(time=1)

ds_diff_k13_dsyr1_lidar = ds_k13_dsyr1['band1']- ds_k13_dsyr1['band1'].isel(time=0)
ds_diff_k13_dsyr1_sfm = ds_k13_dsyr1['band1']- ds_k13_dsyr1['band1'].isel(time=1)

ds_diff_k13_wsyr2_lidar = ds_k13_wsyr2['band1']- ds_k13_wsyr2['band1'].isel(time=0)
ds_diff_k13_wsyr2_sfm = ds_k13_wsyr2['band1']- ds_k13_wsyr2['band1'].isel(time=1)

ds_diff_m14_wsyr1_lidar = ds_m14_wsyr1['band1']- ds_m14_wsyr1['band1'].isel(time=0)
ds_diff_m14_wsyr1_sfm = ds_m14_wsyr1['band1']- ds_m14_wsyr1['band1'].isel(time=1)

ds_diff_m14_dsyr1_lidar = ds_m14_dsyr1['band1']- ds_m14_dsyr1['band1'].isel(time=0)
ds_diff_m14_dsyr1_sfm = ds_m14_dsyr1['band1']- ds_m14_dsyr1['band1'].isel(time=1)

ds_diff_m14_wsyr2_lidar = ds_m14_wsyr2['band1']- ds_m14_wsyr2['band1'].isel(time=0)
ds_diff_m14_wsyr2_sfm = ds_m14_wsyr2['band1']- ds_m14_wsyr2['band1'].isel(time=1)

In [31]:
# ds_diff_k13_wsyr1_lidar.isel(time=1).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();
# ds_diff_k13_wsyr1_sfm.isel(time=0).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();

# ds_diff_k13_dsyr1_lidar.isel(time=1).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();
# ds_diff_k13_dsyr1_sfm.isel(time=0).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();

# ds_diff_k13_wsyr2_lidar.isel(time=1).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();
# ds_diff_k13_wsyr2_sfm.isel(time=0).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();

# ds_diff_m14_wsyr1_lidar.isel(time=1).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();
# ds_diff_m14_wsyr1_sfm.isel(time=0).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();

# ds_diff_m14_dsyr1_lidar.isel(time=1).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();
# ds_diff_m14_dsyr1_sfm.isel(time=0).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();

# ds_diff_m14_wsyr2_lidar.isel(time=1).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();
# ds_diff_m14_wsyr2_sfm.isel(time=0).plot(vmin=-0.05, vmax=0.05, cmap='RdBu')
# plt.show();

## Apply HPGF to stacked dataset

In [32]:
sigma = 50
for i in range(0,5):
    globals()[f'ds_diff_smooth{i}_k13_wsyr1_lidar'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_k13_wsyr1_lidar.isel(time=i).values, sigma = sigma, origmask=True)
    globals()[f'ds_diff_smooth{i}_k13_wsyr1_sfm'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_k13_wsyr1_sfm.isel(time=i).values, sigma = sigma, origmask=True)

ds_diff_k13_wsyr1_lidar = ds_diff_k13_wsyr1_lidar - [ds_diff_smooth0_k13_wsyr1_lidar, \
    ds_diff_smooth1_k13_wsyr1_lidar, ds_diff_smooth2_k13_wsyr1_lidar,\
    ds_diff_smooth3_k13_wsyr1_lidar, ds_diff_smooth4_k13_wsyr1_lidar]

ds_diff_k13_wsyr1_sfm = ds_diff_k13_wsyr1_sfm - [ds_diff_smooth0_k13_wsyr1_sfm, \
    ds_diff_smooth1_k13_wsyr1_sfm, ds_diff_smooth2_k13_wsyr1_sfm,\
    ds_diff_smooth3_k13_wsyr1_sfm, ds_diff_smooth4_k13_wsyr1_sfm]

Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying orig

In [33]:
for i in range(0,4):
    globals()[f'ds_diff_smooth{i}_k13_dsyr1_lidar'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_k13_dsyr1_lidar.isel(time=i).values, sigma = sigma, origmask=True)
    globals()[f'ds_diff_smooth{i}_k13_dsyr1_sfm'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_k13_dsyr1_sfm.isel(time=i).values, sigma = sigma, origmask=True)

ds_diff_k13_dsyr1_lidar = ds_diff_k13_dsyr1_lidar - [ds_diff_smooth0_k13_dsyr1_lidar, \
    ds_diff_smooth1_k13_dsyr1_lidar, ds_diff_smooth2_k13_dsyr1_lidar,\
    ds_diff_smooth3_k13_dsyr1_lidar]

ds_diff_k13_dsyr1_sfm = ds_diff_k13_dsyr1_sfm - [ds_diff_smooth0_k13_dsyr1_sfm, \
    ds_diff_smooth1_k13_dsyr1_sfm, ds_diff_smooth2_k13_dsyr1_sfm,\
    ds_diff_smooth3_k13_dsyr1_sfm]

Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask


In [34]:
for i in range(0,5):
    globals()[f'ds_diff_smooth{i}_k13_wsyr2_lidar'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_k13_wsyr2_lidar.isel(time=i).values, sigma = sigma, origmask=True)
    globals()[f'ds_diff_smooth{i}_k13_wsyr2_sfm'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_k13_wsyr2_sfm.isel(time=i).values, sigma = sigma, origmask=True)

ds_diff_k13_wsyr2_lidar = ds_diff_k13_wsyr2_lidar - [ds_diff_smooth0_k13_wsyr2_lidar, \
    ds_diff_smooth1_k13_wsyr2_lidar, ds_diff_smooth2_k13_wsyr2_lidar,\
    ds_diff_smooth3_k13_wsyr2_lidar, ds_diff_smooth4_k13_wsyr2_lidar]

ds_diff_k13_wsyr2_sfm = ds_diff_k13_wsyr2_sfm - [ds_diff_smooth0_k13_wsyr2_sfm, \
    ds_diff_smooth1_k13_wsyr2_sfm, ds_diff_smooth2_k13_wsyr2_sfm,\
    ds_diff_smooth3_k13_wsyr2_sfm, ds_diff_smooth4_k13_wsyr2_sfm]

Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying orig

In [35]:
sigma = 50
for i in range(0,3):
    globals()[f'ds_diff_smooth{i}_m14_wsyr1_lidar'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_m14_wsyr1_lidar.isel(time=i).values, sigma = sigma, origmask=True)
    globals()[f'ds_diff_smooth{i}_m14_wsyr1_sfm'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_m14_wsyr1_sfm.isel(time=i).values, sigma = sigma, origmask=True)

ds_diff_m14_wsyr1_lidar = ds_diff_m14_wsyr1_lidar - [ds_diff_smooth0_m14_wsyr1_lidar, \
    ds_diff_smooth1_m14_wsyr1_lidar, ds_diff_smooth2_m14_wsyr1_lidar]

ds_diff_m14_wsyr1_sfm = ds_diff_m14_wsyr1_sfm - [ds_diff_smooth0_m14_wsyr1_sfm, \
    ds_diff_smooth1_m14_wsyr1_sfm, ds_diff_smooth2_m14_wsyr1_sfm]

Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask


In [36]:
sigma = 50
for i in range(0,3):
    globals()[f'ds_diff_smooth{i}_m14_dsyr1_lidar'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_m14_dsyr1_lidar.isel(time=i).values, sigma = sigma, origmask=True)
    globals()[f'ds_diff_smooth{i}_m14_dsyr1_sfm'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_m14_dsyr1_sfm.isel(time=i).values, sigma = sigma, origmask=True)

ds_diff_m14_dsyr1_lidar = ds_diff_m14_dsyr1_lidar - [ds_diff_smooth0_m14_dsyr1_lidar, \
    ds_diff_smooth1_m14_dsyr1_lidar, ds_diff_smooth2_m14_dsyr1_lidar]

ds_diff_m14_dsyr1_sfm = ds_diff_m14_dsyr1_sfm - [ds_diff_smooth0_m14_dsyr1_sfm, \
    ds_diff_smooth1_m14_dsyr1_sfm, ds_diff_smooth2_m14_dsyr1_sfm]

Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask


In [37]:
sigma = 50
for i in range(0,7):
    globals()[f'ds_diff_smooth{i}_m14_wsyr2_lidar'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_m14_wsyr2_lidar.isel(time=i).values, sigma = sigma, origmask=True)
    globals()[f'ds_diff_smooth{i}_m14_wsyr2_sfm'], size = micro.hpgf.gauss_fltr_astropy_fft(ds_diff_m14_wsyr2_sfm.isel(time=i).values, sigma = sigma, origmask=True)

ds_diff_m14_wsyr2_lidar = ds_diff_m14_wsyr2_lidar - [ds_diff_smooth0_m14_wsyr2_lidar, \
    ds_diff_smooth1_m14_wsyr2_lidar, ds_diff_smooth2_m14_wsyr2_lidar,\
    ds_diff_smooth3_m14_wsyr2_lidar, ds_diff_smooth4_m14_wsyr2_lidar,\
    ds_diff_smooth5_m14_wsyr2_lidar, ds_diff_smooth6_m14_wsyr2_lidar]

ds_diff_m14_wsyr2_sfm = ds_diff_m14_wsyr2_sfm - [ds_diff_smooth0_m14_wsyr2_sfm, \
    ds_diff_smooth1_m14_wsyr2_sfm, ds_diff_smooth2_m14_wsyr2_sfm,\
    ds_diff_smooth3_m14_wsyr2_sfm, ds_diff_smooth4_m14_wsyr2_sfm,\
    ds_diff_smooth5_m14_wsyr2_sfm, ds_diff_smooth6_m14_wsyr2_sfm]

Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying original mask
Applying gaussian smoothing filter with size 301 and sigma 50.000 (sum 1.000)
Applying orig

## Plot difference maps

In [38]:
# fig, ax = plt.subplots(1, 4, figsize=(15,9))
# ds_extent= ds_diff_k13_wsyr1_lidar.rio.bounds()
# m = ax[0].imshow(ds_diff_k13_wsyr1_lidar.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_k13_wsyr1_lidar.isel(time=1), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('KID-13\n2020-11-09 LiDAR\n2020-11-09 SfM')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_k13_wsyr1_lidar.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('KID-13\n2020-11-09 LiDAR\n2021-02-08 SfM')
# ax[1].set_facecolor('k')

# ax[2].imshow(ds_diff_k13_wsyr1_lidar.isel(time=3), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[2].set_title('KID-13\n2020-11-09 LiDAR\n2021-04-06 SfM')
# ax[2].set_facecolor('k')

# ax[3].imshow(ds_diff_k13_wsyr1_lidar.isel(time=4), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[3].set_title('KID-13\n2020-11-09 LiDAR\n2021-05-13 SfM')
# ax[3].set_facecolor('k')
# plt.show();


# fig, ax = plt.subplots(1, 4, figsize=(15,9))
# ds_extent= ds_diff_k13_wsyr1_sfm.rio.bounds()
# m = ax[0].imshow(ds_diff_k13_wsyr1_sfm.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_k13_wsyr1_sfm.isel(time=0), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('KID-13\n2020-11-09 SfM\n2020-11-09 LiDAR')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_k13_wsyr1_sfm.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('KID-13\n2020-11-09 SfM\n2021-02-08 SfM')
# ax[1].set_facecolor('k')

# ax[2].imshow(ds_diff_k13_wsyr1_sfm.isel(time=3), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[2].set_title('KID-13\n2020-11-09 SfM\n2021-04-06 SfM')
# ax[2].set_facecolor('k')

# ax[3].imshow(ds_diff_k13_wsyr1_sfm.isel(time=4), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[3].set_title('KID-13\n2020-11-09 SfM\n2021-05-13 SfM')
# ax[3].set_facecolor('k')
# plt.show();

In [39]:
# fig, ax = plt.subplots(1, 3, figsize=(13,9))
# ds_extent= ds_diff_k13_dsyr1_lidar.rio.bounds()
# m = ax[0].imshow(ds_diff_k13_dsyr1_lidar.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_k13_dsyr1_lidar.isel(time=1), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('KID-13\n2021-06-04 LiDAR\n2021-06-04 SfM')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_k13_dsyr1_lidar.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('KID-13\n2021-06-04 LiDAR\n2021-08-19 SfM')
# ax[1].set_facecolor('k')

# ax[2].imshow(ds_diff_k13_dsyr1_lidar.isel(time=3), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[2].set_title('KID-13\n2021-06-04 LiDAR\n2021-09-13 SfM')
# ax[2].set_facecolor('k')
# plt.show();


# fig, ax = plt.subplots(1, 3, figsize=(13,9))
# ds_extent= ds_diff_k13_dsyr1_sfm.rio.bounds()
# m = ax[0].imshow(ds_diff_k13_dsyr1_sfm.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_k13_dsyr1_sfm.isel(time=0), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('KID-13\n2021-06-04 SfM\n2021-06-04 LiDAR')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_k13_dsyr1_sfm.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('KID-13\n2021-06-04 SfM\n2021-08-19 SfM')
# ax[1].set_facecolor('k')

# ax[2].imshow(ds_diff_k13_dsyr1_sfm.isel(time=3), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[2].set_title('KID-13\n2021-06-04 SfM\n2021-09-13 SfM')
# ax[2].set_facecolor('k')
# plt.show();

In [40]:
# fig, ax = plt.subplots(1, 4, figsize=(15,9))
# ds_extent= ds_diff_k13_wsyr2_lidar.rio.bounds()
# m = ax[0].imshow(ds_diff_k13_wsyr2_lidar.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_k13_wsyr2_lidar.isel(time=1), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('KID-13\n2021-10-07 LiDAR\n2021-10-07 SfM')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_k13_wsyr2_lidar.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('KID-13\n2021-10-07 LiDAR\n2022-02-08 SfM')
# ax[1].set_facecolor('k')

# ax[2].imshow(ds_diff_k13_wsyr2_lidar.isel(time=3), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[2].set_title('KID-13\n2021-10-07 LiDAR\n2022-05-03 SfM')
# ax[2].set_facecolor('k')

# ax[3].imshow(ds_diff_k13_wsyr2_lidar.isel(time=4), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[3].set_title('KID-13\n2021-10-07 LiDAR\n2022-05-31 SfM')
# ax[3].set_facecolor('k')
# plt.show();


# fig, ax = plt.subplots(1, 4, figsize=(15,9))
# ds_extent= ds_diff_k13_wsyr2_sfm.rio.bounds()
# m = ax[0].imshow(ds_diff_k13_wsyr2_sfm.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_k13_wsyr2_sfm.isel(time=0), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('KID-13\n2021-10-07 SfM\n2021-10-07 LiDAR')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_k13_wsyr2_sfm.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('KID-13\n2021-10-07 SfM\n2022-02-08 SfM')
# ax[1].set_facecolor('k')

# ax[2].imshow(ds_diff_k13_wsyr2_sfm.isel(time=3), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[2].set_title('KID-13\n2021-10-07 SfM\n2022-05-03 SfM')
# ax[2].set_facecolor('k')

# ax[3].imshow(ds_diff_k13_wsyr2_sfm.isel(time=4), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[3].set_title('KID-13\n2021-10-07 SfM\n2022-05-31 SfM')
# ax[3].set_facecolor('k')
# plt.show();

In [41]:
# fig, ax = plt.subplots(1, 2, figsize=(7,9))
# ds_extent= ds_diff_m14_wsyr1_lidar.rio.bounds()
# m = ax[0].imshow(ds_diff_m14_wsyr1_lidar.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_m14_wsyr1_lidar.isel(time=1), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('MEL-14\n2020-12-03 LiDAR\n2020-12-03 SfM')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_m14_wsyr1_lidar.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('MEL-14\n2020-12-03 LiDAR\n2021-02-24 SfM')
# ax[1].set_facecolor('k')
# plt.show();


# fig, ax = plt.subplots(1, 2, figsize=(7,9))
# ds_extent= ds_diff_m14_wsyr1_sfm.rio.bounds()
# m = ax[0].imshow(ds_diff_m14_wsyr1_sfm.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_m14_wsyr1_sfm.isel(time=0), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('MEL-14\n2020-12-03 SfM\n2020-12-03 LiDAR')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_m14_wsyr1_sfm.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('MEL-14\n2020-12-03 SfM\n2021-02-24 SfM')
# ax[1].set_facecolor('k')
# plt.show();

In [42]:
# fig, ax = plt.subplots(1, 2, figsize=(7,9))
# ds_extent= ds_diff_m14_dsyr1_lidar.rio.bounds()
# m = ax[0].imshow(ds_diff_m14_dsyr1_lidar.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_m14_dsyr1_lidar.isel(time=1), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('MEL-14\n2021-06-03 LiDAR\n2021-06-03 SfM')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_m14_dsyr1_lidar.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('MEL-14\n2021-06-03 LiDAR\n2021-09-14 SfM')
# ax[1].set_facecolor('k')
# plt.show();


# fig, ax = plt.subplots(1, 2, figsize=(7,9))
# ds_extent= ds_diff_m14_dsyr1_sfm.rio.bounds()
# m = ax[0].imshow(ds_diff_m14_dsyr1_sfm.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_m14_dsyr1_sfm.isel(time=0), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('MEL-14\n2021-06-03 SfM\n2021-06-03 LiDAR')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_m14_dsyr1_sfm.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('MEL-14\n2021-06-03 SfM\n2021-09-14 SfM')
# ax[1].set_facecolor('k')
# plt.show();

In [43]:
# fig, ax = plt.subplots(1, 6, figsize=(20,9))
# ds_extent= ds_diff_m14_wsyr2_lidar.rio.bounds()
# m = ax[0].imshow(ds_diff_m14_wsyr2_lidar.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_m14_wsyr2_lidar.isel(time=1), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('MEL-14\n2022-03-09 LiDAR\n2022-03-09 SfM')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_m14_wsyr2_lidar.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('MEL-14\n2022-03-09 LiDAR\n2022-03-16 SfM')
# ax[1].set_facecolor('k')

# ax[2].imshow(ds_diff_m14_wsyr2_lidar.isel(time=3), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[2].set_title('MEL-14\n2022-03-09 LiDAR\n2022-03-24 SfM')
# ax[2].set_facecolor('k')

# ax[3].imshow(ds_diff_m14_wsyr2_lidar.isel(time=4), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[3].set_title('MEL-14\n2022-03-09 LiDAR\n2022-04-11 SfM')
# ax[3].set_facecolor('k')

# ax[4].imshow(ds_diff_m14_wsyr2_lidar.isel(time=5), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[4].set_title('MEL-14\n2022-03-09 SfM\n2022-04-28 SfM')
# ax[4].set_facecolor('k')

# ax[5].imshow(ds_diff_m14_wsyr2_lidar.isel(time=6), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[5].set_title('MEL-14\n2022-03-09 SfM\n2022-06-01 SfM')
# ax[5].set_facecolor('k')
# plt.show();


# fig, ax = plt.subplots(1, 6, figsize=(20,9))
# ds_extent= ds_diff_m14_wsyr2_sfm.rio.bounds()
# m = ax[0].imshow(ds_diff_m14_wsyr2_sfm.isel(time=0), cmap = 'RdBu', vmin=-0.025, vmax=0.025)
# fig.colorbar(m, label='Elevation (m)', extend='both', ax=ax)

# ax[0].imshow(ds_diff_m14_wsyr2_sfm.isel(time=0), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[0].set_title('MEL-14\n2022-03-09 SfM\n2022-03-09 LiDAR')
# ax[0].set_facecolor('k')

# ax[1].imshow(ds_diff_m14_wsyr2_sfm.isel(time=2), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[1].set_title('MEL-14\n2022-03-09 SfM\n2022-03-16 SfM')
# ax[1].set_facecolor('k')

# ax[2].imshow(ds_diff_m14_wsyr2_sfm.isel(time=3), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[2].set_title('MEL-14\n2022-03-09 SfM\n2022-03-24 SfM')
# ax[2].set_facecolor('k')

# ax[3].imshow(ds_diff_m14_wsyr2_sfm.isel(time=4), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[3].set_title('MEL-14\n2022-03-09 SfM\n2022-04-11 SfM')
# ax[3].set_facecolor('k')

# ax[4].imshow(ds_diff_m14_wsyr2_sfm.isel(time=5), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[4].set_title('MEL-14\n2022-03-09 SfM\n2022-04-28 SfM')
# ax[4].set_facecolor('k')

# ax[5].imshow(ds_diff_m14_wsyr2_sfm.isel(time=6), cmap='RdBu', vmin=-0.025, vmax=0.025, extent = ds_extent)
# ax[5].set_title('MEL-14\n2022-03-09 SfM\n2022-06-01 SfM')
# ax[5].set_facecolor('k')
# plt.show();

## Raster stats

In [44]:
ds_diff_k13_wsyr1_lidar.to_netcdf('../data/k13_wsyr1_diff.nc')
ds_diff_k13_dsyr1_lidar.to_netcdf('../data/k13_dsyr1_diff.nc')
ds_diff_k13_wsyr2_lidar.to_netcdf('../data/k13_wsyr2_diff.nc')

In [45]:
ds_diff_k13_wsyr1_sfm.to_netcdf('../data/k13_wsyr1_diff_sfm.nc')
ds_diff_k13_dsyr1_sfm.to_netcdf('../data/k13_dsyr1_diff_sfm.nc')
ds_diff_k13_wsyr2_sfm.to_netcdf('../data/k13_wsyr2_diff_sfm.nc')

In [46]:
ds_diff_m14_wsyr1_lidar.to_netcdf('../data/m14_wsyr1_diff.nc')
ds_diff_m14_dsyr1_lidar.to_netcdf('../data/m14_dsyr1_diff.nc')
ds_diff_m14_wsyr2_lidar.to_netcdf('../data/m14_wsyr2_diff.nc')

In [47]:
ds_diff_m14_wsyr1_sfm.to_netcdf('../data/m14_wsyr1_diff_sfm.nc')
ds_diff_m14_dsyr1_sfm.to_netcdf('../data/m14_dsyr1_diff_sfm.nc')
ds_diff_m14_wsyr2_sfm.to_netcdf('../data/m14_wsyr2_diff_sfm.nc')

In [48]:
# pprint(rasterstats.zonal_stats(k13_mask_gdf, 'k13_wsyr1_diff.tif', \
#     stats=['min', 'max', 'median', 'mean', 'std', 'sum']))
# pprint(rasterstats.zonal_stats(k13_mask_gdf, 'k13_dsyr1_diff.tif', \
#     stats=['min', 'max', 'median', 'mean', 'std', 'sum']))
# pprint(rasterstats.zonal_stats(k13_mask_gdf, 'k13_wsyr2_diff.tif', \
#     stats=['min', 'max', 'median', 'mean', 'std', 'sum']))
# pprint(rasterstats.zonal_stats(m14_mask_gdf, 'm14_wsyr1_diff.tif', \
#     stats=['min', 'max', 'median', 'mean', 'std', 'sum']))
# pprint(rasterstats.zonal_stats(m14_mask_gdf, 'm14_dsyr1_diff.tif', \
#     stats=['min', 'max', 'median', 'mean', 'std', 'sum']))
# pprint(rasterstats.zonal_stats(m14_mask_gdf, 'm14_wsyr2_diff.tif', \
#     stats=['min', 'max', 'median', 'mean', 'std', 'sum']))


## Testing other code
Will make a new notebook for this soon.

In [49]:
# #Functions from Friedrich
# import numpy as np
# import pandas as pd
# import geopandas as gpd
# from shapely.geometry import Polygon
# import folium
# from folium.plugins import Draw
# import matplotlib
# import matplotlib.pyplot as plt
# from matplotlib import colormaps
# from mpl_toolkits.axes_grid1 import make_axes_locatable
# from matplotlib.lines import Line2D
# import math

# import pathlib
# import rasterio
# from rasterio.warp import Resampling
# import rioxarray
# import xarray as xr

# def select_points_on_map(gdf, 
#                          output_file_name = 'points.geojson'):
    
#     m = gdf.explore(tiles=None,
#                     style_kwds=dict(fillOpacity=0),
#                     name="polygon")
    
#     folium.TileLayer(
#         "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
#         attr='Google',
#         opacity=0.8,
#         name='Google basemap',
#         max_zoom=25,
#     ).add_to(m)
    

#     Draw(export=True,
#          filename=output_file_name,
#          position = 'topleft').add_to(m)
    
#     folium.LayerControl(collapsed=True).add_to(m)
    
#     minimap = folium.plugins.MiniMap(position="bottomleft",)
#     m.add_child(minimap)

    
#     return m

# def extract_linestring_coords(linestring):
#     """
#     Function to extract x, y coordinates from linestring object
#     Input:
#     shapely.geometry.linestring.LineString
#     Returns:
#     [x: np.array,y: np.array]
#     """
#     x = []
#     y = []
#     for coords in linestring.coords:
#         x.append(coords[0])
#         y.append(coords[1])
#     return [np.array(x), np.array(y)]

In [50]:
# select_points_on_map(k13_mask_gdf, output_file_name='test.geojson')

In [51]:
# scg_points_gdf = gpd.read_file('../data/test.geojson')
# scg_points_gdf.explore()



In [52]:
# test = extract_linestring_coords(scg_points_gdf.to_crs(ds_k13_wsyr1.rio.crs)['geometry'][4])

# test

In [53]:
# ds_k13_wsyr1['band1'].sel(x=test[0], y=test[0], method="nearest").values

In [54]:
# tmp = []
# for i, v in enumerate(test):
#     sub = ds_k13_wsyr1['band1'].isel(time=1).sel(x=test[0], 
#                                        y=test[0], 
#                                        method="backfill")
#     print(i, v)
    
#     tmp.append(sub.values)

# tmp

In [55]:
# plt.plot(test, tmp)